In [1]:
pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
"""
@authors: faurand, chardes, ehagensieker
"""

# all our imports 
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import datetime
import tqdm
import re


# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

In [3]:
# bash code to mount the drive
drive.mount("/content/drive")
os.chdir("drive/MyDrive")

Mounted at /content/drive


In [4]:
# 2.1 the dataset
with open("/content/bible.txt", "r") as f:
  bible = f.read()

# print a short example
print(bible[:500])

The First Book of Moses:  Called Genesis


1:1 In the beginning God created the heaven and the earth.

1:2 And the earth was without form, and void; and darkness was upon
the face of the deep. And the Spirit of God moved upon the face of the
waters.

1:3 And God said, Let there be light: and there was light.

1:4 And God saw the light, that it was good: and God divided the light
from the darkness.

1:5 And God called the light Day, and the darkness he called Night.
And the evening and the mornin


In [5]:
# exersize 2.2
import tensorflow_text as tf_text
# convert to lower case, remove \n and special characters
bible = bible.replace("\n", " ").lower()
bible = re.sub('[^A-Za-z]+', ' ', bible)
# bible = re.sub(r'[0-9]+', lambda m: num2words(m.group(0)), bible) 
print("converted: ", bible[:500])

# tokenize the text
bible = tf_text.WhitespaceTokenizer().split(bible)
print("tokenized: ", bible[:500])

converted:  the first book of moses called genesis in the beginning god created the heaven and the earth and the earth was without form and void and darkness was upon the face of the deep and the spirit of god moved upon the face of the waters and god said let there be light and there was light and god saw the light that it was good and god divided the light from the darkness and god called the light day and the darkness he called night and the evening and the morning were the first day and god said let the
tokenized:  tf.Tensor(
[b'the' b'first' b'book' b'of' b'moses' b'called' b'genesis' b'in' b'the'
 b'beginning' b'god' b'created' b'the' b'heaven' b'and' b'the' b'earth'
 b'and' b'the' b'earth' b'was' b'without' b'form' b'and' b'void' b'and'
 b'darkness' b'was' b'upon' b'the' b'face' b'of' b'the' b'deep' b'and'
 b'the' b'spirit' b'of' b'god' b'moved' b'upon' b'the' b'face' b'of'
 b'the' b'waters' b'and' b'god' b'said' b'let' b'there' b'be' b'light'
 b'and' b'there' b'was' b'light' b'

In [29]:
# counting all words from the corpus and get 10000 most frequent words
words = list(bible.numpy())
counts = Counter(words)
print(counts)

most_common = counts.most_common(10000)
words = [x[0] for x in most_common]
print("most common words: ", words)


Counter({b'the': 64020, b'and': 51696, b'of': 34668, b'to': 13580, b'that': 12912, b'in': 12667, b'he': 10419, b'shall': 9838, b'unto': 8997, b'for': 8971, b'i': 8854, b'his': 8473, b'a': 8177, b'lord': 7964, b'they': 7376, b'be': 7013, b'is': 6989, b'him': 6659, b'not': 6596, b'them': 6430, b'it': 6129, b'with': 6012, b'all': 5620, b'thou': 5474, b'thy': 4600, b'was': 4522, b'god': 4472, b'which': 4413, b'my': 4368, b'me': 4096, b'said': 3999, b'but': 3992, b'ye': 3983, b'their': 3942, b'have': 3904, b'will': 3836, b'thee': 3827, b'from': 3642, b'as': 3520, b'are': 2950, b'when': 2834, b'this': 2785, b'out': 2775, b'were': 2772, b'upon': 2748, b'man': 2735, b'by': 2625, b'you': 2617, b'israel': 2575, b'king': 2541, b'son': 2392, b'up': 2380, b'there': 2299, b'hath': 2264, b'then': 2169, b'people': 2143, b'came': 2093, b'had': 2026, b'house': 2024, b'into': 2015, b'on': 2011, b'her': 1995, b'come': 1971, b'one': 1969, b'we': 1844, b'children': 1821, b'before': 1796, b's': 1790, b'your'

In [40]:
# apply subsampling to discard words that appear very often
s = 0.001
word_counts = Counter(words)
total_count = 1000

freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - (np.sqrt(freqs[word]/s) + 1) for word in word_counts}
train_words = [word for word in words if random.random() < (1 - p_drop[word])]
print(len(train_words), train_words)

10000 [b'the', b'and', b'of', b'to', b'that', b'in', b'he', b'shall', b'unto', b'for', b'i', b'his', b'a', b'lord', b'they', b'be', b'is', b'him', b'not', b'them', b'it', b'with', b'all', b'thou', b'thy', b'was', b'god', b'which', b'my', b'me', b'said', b'but', b'ye', b'their', b'have', b'will', b'thee', b'from', b'as', b'are', b'when', b'this', b'out', b'were', b'upon', b'man', b'by', b'you', b'israel', b'king', b'son', b'up', b'there', b'hath', b'then', b'people', b'came', b'had', b'house', b'into', b'on', b'her', b'come', b'one', b'we', b'children', b'before', b's', b'your', b'also', b'day', b'land', b'an', b'so', b'men', b'against', b'shalt', b'if', b'at', b'let', b'go', b'hand', b'us', b'saying', b'made', b'went', b'no', b'even', b'do', b'now', b'behold', b'saith', b'therefore', b'every', b'these', b'because', b'after', b'our', b'things', b'father', b'down', b'or', b'sons', b'hast', b'o', b'david', b'make', b'say', b'may', b'over', b'did', b'earth', b'what', b'jesus', b'she', b'wh

In [41]:
dict_1 = {x:i for i, x in enumerate(list(train_words))}
print(dict_1)

# input-target pairs
tuples = []
for word in range(1000):
  for neighbor in range(word-2,word+3):
    if word!=neighbor and neighbor>=0 and neighbor<1000:
      tuples.append((dict_1[train_words[word]], dict_1[train_words[neighbor]]))
print(tuples)

{b'the': 0, b'and': 1, b'of': 2, b'to': 3, b'that': 4, b'in': 5, b'he': 6, b'shall': 7, b'unto': 8, b'for': 9, b'i': 10, b'his': 11, b'a': 12, b'lord': 13, b'they': 14, b'be': 15, b'is': 16, b'him': 17, b'not': 18, b'them': 19, b'it': 20, b'with': 21, b'all': 22, b'thou': 23, b'thy': 24, b'was': 25, b'god': 26, b'which': 27, b'my': 28, b'me': 29, b'said': 30, b'but': 31, b'ye': 32, b'their': 33, b'have': 34, b'will': 35, b'thee': 36, b'from': 37, b'as': 38, b'are': 39, b'when': 40, b'this': 41, b'out': 42, b'were': 43, b'upon': 44, b'man': 45, b'by': 46, b'you': 47, b'israel': 48, b'king': 49, b'son': 50, b'up': 51, b'there': 52, b'hath': 53, b'then': 54, b'people': 55, b'came': 56, b'had': 57, b'house': 58, b'into': 59, b'on': 60, b'her': 61, b'come': 62, b'one': 63, b'we': 64, b'children': 65, b'before': 66, b's': 67, b'your': 68, b'also': 69, b'day': 70, b'land': 71, b'an': 72, b'so': 73, b'men': 74, b'against': 75, b'shalt': 76, b'if': 77, b'at': 78, b'let': 79, b'go': 80, b'hand':

In [42]:
# char_tokens = [dict_1[char] for char in tuples]
# dataset = tf.data.Dataset.from_tensor_slices(char_tokens)
dataset = tf.data.Dataset.from_tensor_slices(tuples)

def prepare_data(data, batch_size = 128, sequence_length = 4): 
  data = data.map(lambda x, target: (tf.cast(x, tf.int64), tf.cast(target, tf.int64)))
  #shuffle, batch, prefetch
  data = data.shuffle(10000)
  data = data.batch(batch_size)
  data = data.prefetch(tf.data.AUTOTUNE)
  
  return data

train_ds = prepare_data(dataset)
for seq in train_ds.take(1):
  tf.print(seq)

TypeError: ignored

In [43]:
class SkipGram(tf.keras.layers.Layer): 

  def __init__(self, num_words, embedding): 

    super(SkipGram, self).__init__()
    self.num_words = num_words
    self.embedding = embedding

  def build(self, input_shape): 
    self.weight_matrix = self.add_weight(shape=(self.num_words, self.embedding), trainable = True)

  @tf.function
  def call(self, inputs, labels): 
    embeddings = tf.nn.embedding_lookup(self.weight_matrix, inputs)
    return embeddings
    

In [44]:
def train_step(model, input, target, optimizer = tf.keras.optimizers.SGD(), num_words = 10000): 

  with tf.GradientTape() as tape: 
    embedding = model(input)
    biases = tf.Variable(tf.zeros([num_words]))
    target = tf.reshape(target, (target.shape[0], 1))
    loss = tf.nn.nce_loss(weights = model.get_weights()[0], biases = biases, labels = target, inputs = embedding, num_classes = num_words)

    loss = tf.reduce_mean(loss)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  return loss

def train_step(self, test_ds): 
  losses = []

  for input, target in test_ds: 
    target = tf.expand_dims(target, -1)
    loss = self(input, target)
    losses.append(loss)
    
  return tf.reduce_mean(losses, axis = 0)
  

In [45]:
def training_loop(model, train_ds, test_ds, optimizer, num_words, epochs = 10): 
  
  for epoch in range(epochs): 

    losses = []

    for input, target in train_ds: 
      train_loss = train_step(model, input, target, optimizer, num_words = num_words)
      losses.append(losses)

    value = model.embedding.numpy()
    cosine = tf.keras.losses.CosineSimilarity()

    for data in test_ds: 
      embedding = model(data)
      cosine = np.ones(value.shape[0])
      


In [46]:
text_words = ["holy, father, wine, poison, love, strong, day"]
num_words = 10000
embedding = 64
num_epochs = 15

model = SkipGram(num_words = num_words, embedding = embedding)

